In [6]:
# -*- coding: utf-8 -*-

"""
Created on Mon June 22 10:05:19 2020

@author: Onkar Kulkarni

"""

'\nCreated on Mon June 22 10:05:19 2020\n\n@author: Onkar Kulkarni\n\n'

In [7]:
class MST:
    """ Class to get minimum spanning tree from a graph. """
    def __init__(self, node_count, graph):
        """ initialize nodes and strore graph. """
        self.node_count = node_count
        self.graph = graph
        
        self.triads = []
        for row in range(self.node_count):
            for column in range(self.node_count):
                if self.graph[row][column] != None:
                    if row >= column: continue
                    self.triads.append([row, column, self.graph[row][column]])

        return
    
    
    def getMinCostEdge(self, distance, visited):
        """ Helper function to get the edge such that the edge
        is the minimum among E(visited_nodes, unvisited_nodes)"""
        minDist = float("inf") 
        for v in range(self.node_count): 
            if distance[v] < minDist and not visited[v]: 
                minDist = distance[v] 
                min_index = v 
        return min_index 

    def prims(self): 
        """ 
        For: Undirected and connected graph
        Works by: Adding least cost edge between visited and unvisted
        nodes at each iteration. Does not work on discontinued graph.
        Time = O()
        Space = O()
        """
        distance = [float("inf")] * self.node_count 
        previous = [None] * self.node_count 
        visited = [False] * self.node_count 

        distance[0] = 0 
        previous[0] = -1
  
        for _ in range(self.node_count): 
            current_node = self.getMinCostEdge(distance, visited)  # from unexplored
            visited[current_node] = True
            for adj_node in range(self.node_count): 
                if self.graph[current_node][adj_node] and not visited[adj_node] and distance[adj_node] > self.graph[current_node][adj_node]: 
                    distance[adj_node] = self.graph[current_node][adj_node] 
                    previous[adj_node] = current_node
        
        print (" Edge  \tWeight")
        for i in range(1, self.node_count): 
            print (previous[i], "-", i, "\t", self.graph[i][previous[i]])
        
        return
    
    
    def krushkals(self):
        """
        For: Undirected and connected or disconnected graph
        Works by: selecting least cost edge of remaining graph at 
        each iteration. the edge need not to be connected to already 
        visited nodes. Works on disconnected graphs
        Time = O(ElogV))
        Space = O()
        """
        visited = {}
        unvisited = [x for x in range(self.node_count)]
        mst = []
        
        self.triads = sorted(self.triads, key=lambda item: item[2])     # nlogn time
        index = 0
        while len(visited) < self.node_count:
            source, destination, weight = self.triads[index]
            if source not in visited or destination not in visited:
                visited[source] = destination
                visited[destination] = source
                mst.append([source, destination, weight])
            index += 1
        
        print (" Edge  \tWeight")
        for source, destination, weight in mst: 
            print (source, "-", destination, "\t", weight)
            
    

    def findSet(self, parent, node): 
        """A utility function to find set of a node (path compression technique)."""
        while node != parent[node]:
            node = parent[node]
        return node

    def union(self, parent, rank, source, destination):
        """A function that does union of two sets of sourceSet and set2 (uses union by rank)"""
        sourceSet = self.findSet(parent, source) 
        destinationSet = self.findSet(parent, destination) 

        # Attach smaller rank tree under root of high rank tree Union by Rank 
        if rank[sourceSet] < rank[destinationSet]: 
            parent[sourceSet] = destinationSet 
        elif rank[sourceSet] > rank[destinationSet]: 
            parent[destinationSet] = sourceSet
        else :
            parent[destinationSet] = sourceSet 
            rank[sourceSet] += 1
    
    def boruvkas(self):
        """ 
        For: Undirected and connected or graph
        Works by: combining set of nodes in each iteration. 
        The sets are combined using least cost edge inbetween them.
        Time = O(ElogV))
        Space = O()
        """
        parent = [x for x in range(self.node_count)]
        rank = [0]*self.node_count
        
        # to store index of the cheapest edge of subset. It stores [u,v,w] for each component 
        cheapest = [-1]*self.node_count 
        
        # Initially there are V different trees. 
        # Finally there will be one tree that will be MST 
        numTrees = self.node_count
        mstWeight = 0
        mst = []

        # Keep combining components (or sets) until all 
        # compnentes are not combined into single MST 
        while numTrees > 1: 

            # Traverse through all edges and update cheapest of every component 
            for i in range(len(self.triads)): 

                # findSet components (or sets) of two corner of current edge 
                source,destination,weight =  self.triads[i] 
                sourceSet = self.findSet(parent, source) 
                set2 = self.findSet(parent, destination) 

                # If two corners of current edge belong to 
                # same set, ignore current edge. Else check if  
                # current edge is closer to previous 
                # cheapest edges of sourceSet and set2 
                if sourceSet != set2:      
                    if cheapest[sourceSet] == -1 or cheapest[sourceSet][2] > weight : 
                        cheapest[sourceSet] = [source, destination, weight]
                    if cheapest[set2] == -1 or cheapest[set2][2] > weight : 
                        cheapest[set2] = [source, destination, weight]

            # Consider the above picked cheapest edges and add them 
            # to MST
            for node in range(self.node_count): 
                #Check if cheapest for current set exists 
                if cheapest[node] != -1: 
                    source, destination, weight = cheapest[node] 
                    sourceSet = self.findSet(parent, source) 
                    set2 = self.findSet(parent, destination) 

                    if sourceSet != set2 : 
                        mstWeight += weight 
                        self.union(parent, rank, sourceSet, set2)
                        mst.append([source, destination, weight])
                        numTrees = numTrees - 1

            #reset cheapest array 
            cheapest =[-1] * self.node_count

        print (" Edge  \tWeight")
        for source, destination, weight in mst: 
            print (source, "-", destination, "\t", weight)
        print ("Weight of MST is %d" % mstWeight)

        return
            
    

In [8]:
graph =[[0,    2, None, 6,    None], 
        [2,    0, 3,    8,    5   ], 
        [None, 3, 0,    None, 7   ], 
        [6,    8, None, 0,    9   ], 
        [None, 5, 7,    9,    0   ]] 
mst = MST(5, graph)

print("Prims Algorithm: ")
mst.prims()
print("\n\nKrushkal's Algorithm: ")
mst.krushkals()

print("\n\nBoruvka's Algorithm: ")
mst.boruvkas()

Prims Algorithm: 
 Edge  	Weight
0 - 1 	 2
1 - 2 	 3
0 - 3 	 6
1 - 4 	 5


Krushkal's Algorithm: 
 Edge  	Weight
0 - 1 	 2
1 - 2 	 3
1 - 4 	 5
0 - 3 	 6


Boruvka's Algorithm: 
 Edge  	Weight
0 - 1 	 2
1 - 2 	 3
0 - 3 	 6
1 - 4 	 5
Weight of MST is 16
